In [ ]:
import os
from dotenv import load_dotenv


from langchain_community.vectorstores.azuresearch import AzureSearch
from langchain_openai import OpenAIEmbeddings

from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader

load_dotenv()

True

In [ ]:
# Settings
vector_store_address = os.environ.get("YOUR_AZURE_SEARCH_ENDPOINT")
vector_store_password = os.environ.get("YOUR_AZURE_SEARCH_ADMIN_KEY")
openai_api_key = os.environ.get("OPENAI_API_KEY")

In [ ]:
# Option 1: Use OpenAIEmbeddings with OpenAI account for  embeddings 
model: str = "text-embedding-ada-002"

embeddings: OpenAIEmbeddings = OpenAIEmbeddings(
    openai_api_key=openai_api_key, model=model
)

In [ ]:
# vector store instances
index_name: str = "expotech25_v1"

vector_store: AzureSearch = AzureSearch(
    azure_search_endpoint=vector_store_address,
    azure_search_key=vector_store_password,
    index_name=index_name,
    embedding_function=embeddings.embed_query,
)

In [ ]:
# Insert text and embeddings into vector store
sources = [
    "./retriever_documents/informacion_general_11_24.pdf",
]
def load_pdf(sources):
    docs = []

    for source in sources:
        loaders = [PyPDFLoader(source)]
        for loader in loaders:
            docs.extend(loader.load())

    return docs

documents = load_pdf(sources)

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

vector_store.add_documents(documents=docs)


['NjhmODdlODMtM2FmNy00MTE3LWI5OGEtM2ViMzA3ZDYzZGMy',
 'OWEwYWE4MjgtZWNiYS00YzlkLTlmMWQtY2Q3ODU2MjY0ZmMx',
 'NDJkMWY5YzQtZmQ2MC00MmU1LWE1OWMtZjVjYWQwZjFlOTI2',
 'NmVjNDlmZmYtOTUwNy00MzE4LWJiMTgtYWU5MzA0ZDVmNjEz']

In [ ]:
# Perform a vector similarity search
docs = vector_store.similarity_search(
    query="cuales son las ventajas de participar de la expotech?",
    k=3,
    search_type="similarity",
)
print(docs[0].page_content)

5. Actualización en tendencias tecnológicas: Puedes conocer las innovaciones y 
desarrollos recientes en el sector, manteniéndote al día con los avances 
tecnológicos y aplicando ese conocimiento en tu negocio. 
6. Generación de oportunidades de negocio: La EXPO TECH está diseñada para 
ayudar a las empresas a encontrar nuevos socios y clientes. Las oportunidades de 
colaboración y las ventas pueden surgir durante o después del evento. 
 
Planes de Exhibición Disponibles: 
1. Plan A: 
o Inversión: $1,200.00 + ITBMS 
o Incluye un espacio físico de 3 metros x 2 metros, salida eléctrica hasta 500 
watts, seguridad general, aire acondicionado, iluminación general y aseo 
general. 
2. Plan B: 
o Inversión: $1,500.00 + ITBMS 
o Incluye un stand de 3 metros x 2 metros con paredes blancas de melamina, 
una mesa con dos sillas y cesto de basura, toma corriente hasta 500 watts, 
iluminación con interruptor, rótulo con el nombre de la empresa en la fascia, 
aire acondicionado, iluminación general

In [ ]:
# Perform a vector similarity search with relevance scores

docs_and_scores = vector_store.similarity_search_with_relevance_scores(
    query="cuales son las ventajas de participar de la expotech?",
    k=4,
    score_threshold=0.80,
)
from pprint import pprint

pprint(docs_and_scores)

[(Document(metadata={'id': 'OWEwYWE4MjgtZWNiYS00YzlkLTlmMWQtY2Q3ODU2MjY0ZmMx', 'source': './retriever_documents/informacion_general_11_24.pdf', 'page': 1}, page_content='5. Actualización en tendencias tecnológicas: Puedes conocer las innovaciones y \ndesarrollos recientes en el sector, manteniéndote al día con los avances \ntecnológicos y aplicando ese conocimiento en tu negocio. \n6. Generación de oportunidades de negocio: La EXPO TECH está diseñada para \nayudar a las empresas a encontrar nuevos socios y clientes. Las oportunidades de \ncolaboración y las ventas pueden surgir durante o después del evento. \n \nPlanes de Exhibición Disponibles: \n1. Plan A: \no Inversión: $1,200.00 + ITBMS \no Incluye un espacio físico de 3 metros x 2 metros, salida eléctrica hasta 500 \nwatts, seguridad general, aire acondicionado, iluminación general y aseo \ngeneral. \n2. Plan B: \no Inversión: $1,500.00 + ITBMS \no Incluye un stand de 3 metros x 2 metros con paredes blancas de melamina, \nuna mesa

In [ ]:
# Perform a hybrid search using the search_type parameter
docs = vector_store.similarity_search(
    query="cuales son las ventajas de participar de la expotech?",
    k=3,
    search_type="hybrid",
)
print(docs[0].page_content)

5. Actualización en tendencias tecnológicas: Puedes conocer las innovaciones y 
desarrollos recientes en el sector, manteniéndote al día con los avances 
tecnológicos y aplicando ese conocimiento en tu negocio. 
6. Generación de oportunidades de negocio: La EXPO TECH está diseñada para 
ayudar a las empresas a encontrar nuevos socios y clientes. Las oportunidades de 
colaboración y las ventas pueden surgir durante o después del evento. 
 
Planes de Exhibición Disponibles: 
1. Plan A: 
o Inversión: $1,200.00 + ITBMS 
o Incluye un espacio físico de 3 metros x 2 metros, salida eléctrica hasta 500 
watts, seguridad general, aire acondicionado, iluminación general y aseo 
general. 
2. Plan B: 
o Inversión: $1,500.00 + ITBMS 
o Incluye un stand de 3 metros x 2 metros con paredes blancas de melamina, 
una mesa con dos sillas y cesto de basura, toma corriente hasta 500 watts, 
iluminación con interruptor, rótulo con el nombre de la empresa en la fascia, 
aire acondicionado, iluminación general

In [17]:
# Perform a hybrid search using the hybrid_search method
docs = vector_store.hybrid_search(
    query="cuales son las ventajas de participar de la expotech?", k=3
)
print(docs[0].page_content)

5. Actualización en tendencias tecnológicas: Puedes conocer las innovaciones y 
desarrollos recientes en el sector, manteniéndote al día con los avances 
tecnológicos y aplicando ese conocimiento en tu negocio. 
6. Generación de oportunidades de negocio: La EXPO TECH está diseñada para 
ayudar a las empresas a encontrar nuevos socios y clientes. Las oportunidades de 
colaboración y las ventas pueden surgir durante o después del evento. 
 
Planes de Exhibición Disponibles: 
1. Plan A: 
o Inversión: $1,200.00 + ITBMS 
o Incluye un espacio físico de 3 metros x 2 metros, salida eléctrica hasta 500 
watts, seguridad general, aire acondicionado, iluminación general y aseo 
general. 
2. Plan B: 
o Inversión: $1,500.00 + ITBMS 
o Incluye un stand de 3 metros x 2 metros con paredes blancas de melamina, 
una mesa con dos sillas y cesto de basura, toma corriente hasta 500 watts, 
iluminación con interruptor, rótulo con el nombre de la empresa en la fascia, 
aire acondicionado, iluminación general